# Adaptive Gaussian Mixture Model for Background Subtraction

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt

from cvloop import cvloop
from cvloop.functions import *

### Webcam Feeds

In [ ]:
# Mixture of Gaussians (on Webcam, KaewTraKulPong/Bowden 2001)
cvloop(function=cv_foreground_extractor_mog, side_by_side=True)

In [ ]:
# Mixture of Gaussians (on Webcam, Zivkovic 2004/2006)
cvloop(function=cv_foreground_extractor_mog2, side_by_side=True)

### Test Videos
- [OpenCV Examples](https://github.com/opencv/opencv_extra/tree/7054d3f5d957c8f4849aea34aa1dcbadf4207a38/testdata/cv/video)
- [AVSS 2007 Examples](http://www.eecs.qmul.ac.uk/~andrea/avss2007_d.html)

In [ ]:
videos_cv = ['768x576.avi', '1920x1080.avi'] # OpenCV Examples
videos_ab = ['AVSS_AB_Easy_Divx.avi', 'AVSS_AB_Medium_Divx.avi', 'AVSS_AB_Hard_Divx.avi'] # AVSS 2007
videos_pv = ['AVSS_PV_Easy_Divx.avi', 'AVSS_PV_Medium_Divx.avi', 'AVSS_PV_Hard_Divx.avi', 'AVSS_PV_Night_Divx.avi'] # AVSS 2007

In [ ]:
cvloop(videos_cv[0], function=cv_foreground_extractor_mog2)

### Store N-th frame for comparison

The following uses a sample of the videos and stores the algorithm applications for some frames. This is used for our report.

In [ ]:
class ExtractNthFrame:
    def __init__(self, frameNo, funs, prefix):
        """Initializes to extract frame frameNo."""
        self.frameNo = frameNo
        self.current = 0
        self.funs = funs
        self.prefix = prefix


    def __call__(self, frame):
        """Applies all functions and if frameNo equals current frame, stores all images. 
        Returns the original frame."""
        self.current += 1
        for name, fun in self.funs.items():
            out = fun(frame)
            if self.current == self.frameNo:
                plt.imsave('{}_{}_f{}.png'.format(self.prefix, name, self.current), out, cmap='gray' if 'mask' in name else None)
        if self.current == self.frameNo:
            plt.imsave('{}_original_f{}.png'.format(self.prefix, self.current), frame)
            print('Saved frame {}'.format(self.current))
        return cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)


functions = {
    'mask_mog': cv_background_subtractor_mog,
    'mask_mog2': cv_background_subtractor_mog2,
    'fg_mog': cv_foreground_extractor_mog,
    'fg_mog2': cv_foreground_extractor_mog2
}

In [ ]:
cvloop(videos_cv[0], function=ExtractNthFrame(100, functions, videos_cv[0].split('.')[0]))

In [ ]:
cvloop(videos_cv[1], function=ExtractNthFrame(100, functions, videos_cv[1].split('.')[0]))

In [ ]:
cvloop(videos_pv[2], function=ExtractNthFrame(500, functions, videos_pv[2].split('.')[0]))

In [ ]:
cvloop(videos_pv[3], function=ExtractNthFrame(500, functions, videos_pv[3].split('.')[0]))

In [ ]:
cvloop(videos_ab[0], function=ExtractNthFrame(500, functions, videos_ab[0].split('.')[0]))

In [ ]:
cvloop(videos_ab[2], function=ExtractNthFrame(500, functions, videos_ab[2].split('.')[0]))

### Two mask comparison

Reads the mask files for a given image and creates a side-by-side comparison image.

In [ ]:
image = '768x576'
frame = 100
what = 'mask'
width = 396

name_mog = '{}_{}_mog_f{}.png'.format(image, what, frame)
name_mog2 = '{}_{}_mog2_f{}.png'.format(image, what, frame)

img_mog = plt.imread(name_mog)
img_mog2 = plt.imread(name_mog2)

img_comp = np.hstack((img_mog[:,:width], img_mog2[:,width:]))
img_comp[:,width] = (1, 0, 0, 1)
img_comp = img_comp[100:370,250:680]
plt.figure()
plt.title('Splitting at pixel {}'.format(width))
plt.imshow(img_comp)
plt.imsave('{}_comp_{}_f{}.png'.format(image, what, frame), img_comp)